<a href="https://colab.research.google.com/github/marcelatorrelledo/Algorithms-Section-3-/blob/main/algorithm_personalized_tutor_aid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Original code

from google.colab import drive
import json

drive.mount('/content/drive')

FILE_PATH = '/content/drive/MyDrive/professor_data.json'

class ProfessorRegistry:
    def __init__(self):
        self.professors = {}

    def add_professor(self, professor_name, language, subjects, level):
        professor_id = len(self.professors) + 1
        self.professors[professor_id] = {'name': professor_name, 'language': language, 'subjects': subjects, 'level': level}

    def get_professor_input(self):
        professor_name = input("Enter your name: ")
        language = input("Enter the language you will teach in (English, Spanish, French): ").capitalize()

        while True:
            subjects = input("Enter the subjects you will teach (comma-separated): ").split(',')
            if subjects:
                break
            else:
                print("Please enter at least one subject.")

        while True:
            level = input("Enter your proficiency level in the subjects (low, medium, advanced): ").capitalize()
            if level in ['Low', 'Medium', 'Advanced']:
                break
            else:
                print("Invalid proficiency level. Please enter 'Low', 'Medium', or 'Advanced'.")

        return {'professor_name': professor_name, 'language': language.strip().capitalize(), 'subjects': [s.strip().capitalize() for s in subjects], 'level': level.strip().capitalize()}

    def save_data(self):
        with open(FILE_PATH, 'w') as file:
            json.dump(self.professors, file)

    def load_data(self):
        try:
            with open(FILE_PATH, 'r') as file:
                self.professors = json.load(file)
        except FileNotFoundError:
            pass
        except json.JSONDecodeError:
            print("Error decoding JSON. File may be corrupted.")

    def filter_professors_by_subject(self, subject):
        return {professor_id: professor_info for professor_id, professor_info in self.professors.items() if subject in professor_info['subjects']}

    def calculate_scores(self, student_input):
        filtered_professors = self.filter_professors_by_subject(student_input['subject'])

        scores = {}
        for professor_id, professor_info in filtered_professors.items():
            score = 0
            if student_input['language'] == professor_info['language']:
                score += 15
            if student_input['level'] == professor_info['level']:
                score += 15
            score += 70  # Score for subject is already checked in the filtering step
            scores[professor_id] = score
        return scores

    def display_top_scores(self, top_n=3):
        sorted_scores = sorted(self.calculate_scores(student_input).items(), key=lambda x: x[1], reverse=True)
        top_scores = sorted_scores[:top_n]
        print(f"Top {top_n} Scores:")
        for professor_id, score in top_scores:
            professor_info = self.professors[professor_id]
            print(f"Professor '{professor_info['name']}': Score {score}")

class PersonalizedTutorAid:
    def __init__(self, professor_registry):
        self.professor_registry = professor_registry

    def get_student_input(self):
        language = input("Enter the language you want the subject to be taught in: ").capitalize()
        subject = input("Enter the subject you are looking for aid in: ").capitalize()

        while True:
            level = input("Enter your proficiency level in the subject (low, medium, advanced): ").capitalize()
            if level in ['Low', 'Medium', 'Advanced']:
                break
            else:
                print("Invalid proficiency level. Please enter 'Low', 'Medium', or 'Advanced'.")

        return {'language': language, 'subject': subject, 'level': level}

    def match_tutor_for_student(self, student_input):
        scores = self.professor_registry.calculate_scores(student_input)

        # Get the top 3 scorers
        top_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]

        if not top_scores:
            raise ValueError("No matching professors found for the student's criteria.")
        else:
            # Check if the number of matching professors is less than 3
            if len(top_scores) < 3:
                print("Matching professors:")
                for professor_id, score in top_scores:
                    professor_info = self.professor_registry.professors[professor_id]
                    print(f"Professor {professor_info['name']} teaches {student_input['level']} {student_input['subject']} level in {professor_info['language']}. Score: {score}")

                print("There are no more professors that teach this subject.")
            else:
                # Generate the match output for the top 3
                match_output = []
                for professor_id, score in top_scores:
                    professor_info = self.professor_registry.professors[professor_id]
                    match_output.append(f"The professor {professor_info['name']} teaches {student_input['level']} {student_input['subject']} level in {professor_info['language']}. Score: {score}")

                print("Top 3 matching professors:\n" + "\n".join(match_output))

# Main Program
professor_registry = ProfessorRegistry()
professor_registry.load_data()
pta = PersonalizedTutorAid(professor_registry)

# Inform the user about the file location
print(f"Your data will be saved in: {FILE_PATH}")

user_type = input("Are you a professor or a student? ").lower()

if user_type == 'professor':
    while True:
        add_professor = input("Do you want to add a new professor? (yes/no): ").lower()
        if add_professor != 'yes':
            break

        professor_info = professor_registry.get_professor_input()
        professor_registry.add_professor(**professor_info)

    professor_registry.save_data()

elif user_type == 'student':
    # Get student input
    student_input = pta.get_student_input()

    # Matching tutors for the student based on input
    try:
        result = pta.match_tutor_for_student(student_input)
        # Removed the line to display professors
        # professor_registry.display_top_scores()
    except ValueError as e:
        print(e)
else:
    print("Invalid input. Please enter either 'professor' or 'student'.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Your data will be saved in: /content/drive/MyDrive/professor_data.json
Are you a professor or a student? student
Enter the language you want the subject to be taught in: history
Enter the subject you are looking for aid in: history
Enter your proficiency level in the subject (low, medium, advanced): medium
Top 3 matching professors:
The professor Elena teaches Medium History level in English. Score: 85
The professor noemi teaches Medium History level in French. Score: 85
The professor Antonio teaches Medium History level in Spanish. Score: 85


In [ ]:
## Code after Yan's and Ana's implementation

from google.colab import drive
import json

drive.mount('/content/drive')

FILE_PATH = '/content/drive/MyDrive/professor_data.json'

class ProfessorRegistry:
    def __init__(self):
        self.professors = {}

    def add_professor(self, professor_name, language, subjects, level, email, phone_number):
        professor_id = len(self.professors) + 1
        self.professors[professor_id] = {
            'name': professor_name,
            'language': language,
            'subjects': subjects,
            'level': level,
            'email': email,
            'phone_number': phone_number
        }

    def get_professor_input(self):
        professor_name = input("Enter your name: ")
        language = input("Enter the language you will teach in (English, Spanish, French): ").capitalize()

        while True:
            subjects = input("Enter the subjects you will teach (comma-separated): ").split(',')
            if subjects:
                break
            else:
                print("Please enter at least one subject.")

        while True:
            level = input("Enter your proficiency level in the subjects (low, medium, advanced): ").capitalize()
            if level in ['Low', 'Medium', 'Advanced']:
                break
            else:
                print("Invalid proficiency level. Please enter 'Low', 'Medium', or 'Advanced'.")

        # New: Ask for contact information
        email = input("Enter your email address: ")
        phone_number = input("Enter your phone number: ")

        return {
            'professor_name': professor_name,
            'language': language.strip().capitalize(),
            'subjects': [s.strip().capitalize() for s in subjects],
            'level': level.strip().capitalize(),
            'email': email,
            'phone_number': phone_number
        }

    def save_data(self):
        with open(FILE_PATH, 'w') as file:
            json.dump(self.professors, file)

    def load_data(self):
        try:
            with open(FILE_PATH, 'r') as file:
                self.professors = json.load(file)
        except FileNotFoundError:
            pass
        except json.JSONDecodeError:
            print("Error decoding JSON. File may be corrupted.")

    def filter_professors_by_subject(self, subject):
        return {professor_id: professor_info for professor_id, professor_info in self.professors.items() if subject in professor_info['subjects']}

    def calculate_scores(self, student_input):
        filtered_professors = self.filter_professors_by_subject(student_input['subject'])

        scores = {}
        for professor_id, professor_info in filtered_professors.items():
            score = 0
            if student_input['language'] == professor_info['language']:
                score += 15
            if student_input['level'] == professor_info['level']:
                score += 15
            score += 70  # Score for subject is already checked in the filtering step
            scores[professor_id] = score
        return scores

    def display_top_scores(self, top_n=3):
        print("The score is computed with a 60% weight for the subject, 25 for the level, and 15 for the language.")
        sorted_scores = sorted(self.calculate_scores(student_input).items(), key=lambda x: x[1], reverse=True)
        top_scores = sorted_scores[:top_n]
        print(f"\nTop {top_n} Scores:")
        for professor_id, score in top_scores:
            professor_info = self.professors[professor_id]
            print(f"Professor '{professor_info['name']}': Score {score}")

class PersonalizedTutorAid:
    def __init__(self, professor_registry):
        self.professor_registry = professor_registry

    def get_student_input(self):
        language = input("Enter the language you want the subject to be taught in: ").capitalize()
        subject = input("Enter the subject you are looking for aid in: ").capitalize()

        while True:
            level = input("Enter your proficiency level in the subject (low, medium, advanced): ").capitalize()
            if level in ['Low', 'Medium', 'Advanced']:
                break
            else:
                print("Invalid proficiency level. Please enter 'Low', 'Medium', or 'Advanced'.")

        return {'language': language, 'subject': subject, 'level': level}

    def match_tutor_for_student(self, student_input):
        scores = self.professor_registry.calculate_scores(student_input)

        # Get the top 3 scorers
        top_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]

        print("The score is computed with a 60% weight for the subject, 25 for the level, and 15 for the language.")

        if not top_scores:
            raise ValueError("No matching professors found for the student's criteria.")
        else:
            # Check if the number of matching professors is less than 3
            if len(top_scores) < 3:
                print("\nMatching professors:")
                for professor_id, score in top_scores:
                    professor_info = self

# Main Program
professor_registry = ProfessorRegistry()
professor_registry.load_data()
pta = PersonalizedTutorAid(professor_registry)

# Inform the user about the file location
print(f"Your data will be saved in: {FILE_PATH}")

user_type = input("Are you a professor or a student? ").lower()

if user_type == 'professor':
    while True:
        add_professor = input("Do you want to add a new professor? (yes/no): ").lower()
        if add_professor != 'yes':
            break

        professor_info = professor_registry.get_professor_input()
        professor_registry.add_professor(**professor_info)

    professor_registry.save_data()

elif user_type == 'student':
    # Get student input
    student_input = pta.get_student_input()

    # Matching tutors for the student based on input
    try:
        result = pta.match_tutor_for_student(student_input)
        # Removed the line to display professors
        # professor_registry.display_top_scores()
    except ValueError as e:
        print(e)
else:
    print("Invalid input. Please enter either 'professor' or 'student'.")

In [ ]:
## Changes after Ana's, Carlota's and Yan's implementation

from google.colab import drive
import json

drive.mount('/content/drive')

FILE_PATH = '/content/drive/MyDrive/professor_data.json'

class ProfessorRegistry:
    def __init__(self):
        self.professors = {}

    def add_professor(self, professor_name, language, subjects, level, email=None, phone_number=None):
        professor_id = len(self.professors) + 1
        self.professors[professor_id] = {
            'name': professor_name,
            'language': language,
            'subjects': subjects,
            'level': level,
            'email': email,
            'phone_number': phone_number
        }

    def get_professor_input(self):
        professor_name = input("Enter your name: ")
        language = input("Enter the language you will teach in (English, Spanish, French): ").capitalize()

        while True:
            subjects = input("Enter the subjects you will teach (comma-separated): ").split(',')
            if subjects:
                break
            else:
                print("Please enter at least one subject.")

        while True:
            level = input("Enter your proficiency level in the subjects (low, medium, advanced): ").capitalize()
            if level in ['Low', 'Medium', 'Advanced']:
                break
            else:
                print("Invalid proficiency level. Please enter 'Low', 'Medium', or 'Advanced'.")

        # New: Ask for contact information
        email = input("Enter your email address (optional, press Enter to skip): ").strip()
        phone_number = input("Enter your phone number (optional, press Enter to skip): ").strip()

        return {
            'professor_name': professor_name,
            'language': language.strip().capitalize(),
            'subjects': [s.strip().capitalize() for s in subjects],
            'level': level.strip().capitalize(),
            'email': email if email else None,
            'phone_number': phone_number if phone_number else None
        }

    def save_data(self):
        with open(FILE_PATH, 'w') as file:
            json.dump(self.professors, file)

    def load_data(self):
        try:
            with open(FILE_PATH, 'r') as file:
                self.professors = json.load(file)
        except FileNotFoundError:
            pass
        except json.JSONDecodeError:
            print("Error decoding JSON. File may be corrupted.")

    def filter_professors_by_subject(self, subject):
        return {professor_id: professor_info for professor_id, professor_info in self.professors.items() if subject in professor_info['subjects']}

    def calculate_scores(self, student_input):
        filtered_professors = self.filter_professors_by_subject(student_input['subject'])

        scores = {}
        for professor_id, professor_info in filtered_professors.items():
            score = 0
            if student_input['language'] == professor_info['language']:
                score += 15
            if student_input['level'] == professor_info['level']:
                score += 15
            score += 70  # Score for subject is already checked in the filtering step
            scores[professor_id] = score
        return scores

    def display_top_scores(self, top_n=3):
        print("The score is computed with a 60% weight for the subject, 25 for the level, and 15 for the language.")
        sorted_scores = sorted(self.calculate_scores(student_input).items(), key=lambda x: x[1], reverse=True)
        top_scores = sorted_scores[:top_n]
        print(f"\nTop {top_n} Scores:")
        for professor_id, score in top_scores:
            professor_info = self.professors[professor_id]
            print(f"Professor '{professor_info['name']}': Score {score}")

class PersonalizedTutorAid:
    def __init__(self, professor_registry):
        self.professor_registry = professor_registry

    def get_student_input(self):
        language = input("Enter the language you want the subject to be taught in: ").capitalize()
        subject = input("Enter the subject you are looking for aid in: ").capitalize()

        while True:
            level = input("Enter your proficiency level in the subject (low, medium, advanced): ").capitalize()
            if level in ['Low', 'Medium', 'Advanced']:
                break
            else:
                print("Invalid proficiency level. Please enter 'Low', 'Medium', or 'Advanced'.")

        return {'language': language, 'subject': subject, 'level': level}

    def match_tutor_for_student(self, student_input):
        scores = self.professor_registry.calculate_scores(student_input)

        # Get the top 3 scorers
        top_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]

        print("The score is computed with a 60% weight for the subject, 25 for the level, and 15 for the language.")

        if not top_scores:
            raise ValueError("No matching professors found for the student's criteria.")
        else:
            # Check if the number of matching professors is less than 3
            if len(top_scores) < 3:
                print("\nMatching professors:")
                for professor_id, score in top_scores:
                    professor_info = self.professor_registry.professors[professor_id]
                    print(f"Professor {professor_info['name']} teaches {student_input['level']} {student_input['subject']} level in {professor_info['language']}. Score: {score}")

                    # Display contact information if available
                    email = professor_info.get('email')
                    phone_number = professor_info.get('phone_number')
                    if email and phone_number:
                        print(f"Contact Information: Email - {email}, Phone - {phone_number}")
                    else:
                        print("Contact information not available.")

                print("There are no more professors that teach this subject.")
            else:
                # Generate the match output for the top 3
                match_output = []
                for professor_id, score in top_scores:
                    professor_info = self.professor_registry.professors[professor_id]
                    match_output.append(f"The professor {professor_info['name']} teaches {student_input['level']} {student_input['subject']} level in {professor_info['language']}. Score: {score}")

                    # Include contact information if available
                    email = professor_info.get('email')
                    phone_number = professor_info.get('phone_number')
                    if email and phone_number:
                        match_output.append(f"Contact Information: Email - {email}, Phone - {phone_number}")
                    else:
                        match_output.append("Contact information not available.")

                print("Top 3 matching professors:\n" + "\n".join(match_output))

# Main Program
professor_registry = ProfessorRegistry()
professor_registry.load_data()
pta = PersonalizedTutorAid(professor_registry)

# Inform the user about the file location
print(f"Your data will be saved in: {FILE_PATH}")

user_type = input("Are you a professor or a student? ").lower()

if user_type == 'professor':
    while True:
        add_professor = input("Do you want to add a new professor? (yes/no): ").lower()
        if add_professor != 'yes':
            break

        professor_info = professor_registry.get_professor_input()
        professor_registry.add_professor(**professor_info)

    professor_registry.save_data()

elif user_type == 'student':
    # Get student input
    student_input = pta.get_student_input()

    # Matching tutors for the student based on input
    try:
        result = pta.match_tutor_for_student(student_input)
        # Removed the line to display professors
        # professor_registry.display_top_scores()
    except ValueError as e:
        print(e)
else:
    print("Invalid input. Please enter either 'professor' or 'student'.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Your data will be saved in: /content/drive/MyDrive/professor_data.json
Are you a professor or a student? student
Enter the language you want the subject to be taught in: spanish
Enter the subject you are looking for aid in: mathematics
Enter your proficiency level in the subject (low, medium, advanced): advanced
The score is computed with a 60% weight for the subject, 25 for the level, and 15 for the language.
Top 3 matching professors:
The professor John Doe teaches Advanced Mathematics level in English. Score: 85
Contact information not available.
The professor John Doe teaches Advanced Mathematics level in English. Score: 85
Contact information not available.
The professor Philip teaches Advanced Mathematics level in Spanis. Score: 70
Contact Information: Email - philip@gmail.com, Phone - +34 65814522
